In [14]:
import os
import random
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from skimage.transform import rotate
from skimage.feature import canny
from scipy.stats import zscore
from skimage.measure import regionprops, label
!pip install opencv-python
import cv2
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
import glob

# Root directory containing the image folders
root_dir = '/Users/YK/Desktop'

# Predefined dictionary of date_medium and corresponding heights
data = {
    "24_06_28_water_air_late_cam2": ["3mm","4mm","5mm","air"]
    # "6.19_helium": ["4mm", "air"]
}

image_size = (200, 200)  # Size of each main cell in the grid
trial_image_size = (image_size[0] // 3, image_size[1] // 2)  # Size of each trial image in the 2x3 sub-grid
padding = 10  # Padding between images
font_size = 20  # Font size for labels

# Function to extract numerical value from filename
def extract_number(filename):
    try:
        return int(''.join(filter(str.isdigit, filename.split('_')[0])))
    except ValueError:
        return float('inf')

# Function to organize images for a specific date_medium
def organize_images(date_medium, heights):
    medium_path = os.path.join(root_dir, date_medium)
    
    # Initialize list for delays
    delays = set()

    # Collect all unique delays
    for height in heights:
        jpeg_path = os.path.join(medium_path, height, 'JPEG')
        if os.path.isdir(jpeg_path):
            for file in os.listdir(jpeg_path):
                if file.endswith('.jpg'):
                    delay = file.split('_')[0]
                    delays.add(delay)

    delays = sorted(delays, key=extract_number)

    # Create a blank image with a white background
    grid_width = len(delays)
    grid_height = len(heights)
    total_width = grid_width * (image_size[0] + padding) + padding
    total_height = grid_height * (image_size[1] + padding) + padding + padding  
    grid_image = Image.new('RGB', (total_width, total_height), (255, 255, 255))
    draw = ImageDraw.Draw(grid_image)
    font = ImageFont.load_default()  
    # Add headers
    for col, delay in enumerate(delays):
        x = col * (image_size[0] + padding) + padding + image_size[0] // 2
        draw.text((x, padding // 2), delay, fill="black", font=font, anchor="mm")

    for row, height in enumerate(heights):
        y = row * (image_size[1] + padding) + padding + image_size[1] // 2 + padding
        draw.text((padding // 2, y), height, fill="black", font=font, anchor="mm")

    # Load and place images
    for row, height in enumerate(heights):
        for col, delay in enumerate(delays):
            jpeg_path = os.path.join(medium_path, height, 'JPEG')
            trial_images = [file for file in sorted(os.listdir(jpeg_path)) if file.startswith(delay) and file.endswith('.jpg')]
            if trial_images:
                sub_grid = Image.new('RGB', image_size, (255, 255, 255))  # Sub-grid to hold the 2x3 arrangement of trials
                for idx, trial_image in enumerate(trial_images[:5]):  # Limit to 5 images for the 2x3 grid
                    image_path = os.path.join(jpeg_path, trial_image)
                    if os.path.exists(image_path):
                        img = Image.open(image_path)
                        img = img.resize(trial_image_size, Image.Resampling.LANCZOS)
                        x = (idx % 3) * trial_image_size[0]
                        y = (idx // 3) * trial_image_size[1]
                        sub_grid.paste(img, (x, y))
                x = col * (image_size[0] + padding) + padding
                y = row * (image_size[1] + padding) + padding + padding  # Additional padding for headers
                grid_image.paste(sub_grid, (x, y))


    # Save the organized image
    output_image = f'/Users/YK/Desktop/{date_medium}_organized.jpg'
    grid_image.save(output_image)
    print(f'Saved organized image for {date_medium} as {output_image}')

# Process each date_medium directory
for date_medium, heights in data.items():
    organize_images(date_medium, heights)



Saved organized image for 24_06_28_water_air_late_cam2 as /Users/YK/Desktop/24_06_28_water_air_late_cam2_organized.jpg
